In [1]:
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
segment3_workflow1=pd.read_csv('segment3_workflowS1.csv')
segment3_workflow1.CASE_ID.mode()

0    1175521|568653|866936
dtype: object

In [3]:
segment3_workflow2=pd.read_csv('segment3_workflowS2.csv')
segment3_workflow2.CASE_ID.mode()

0    1114684|484215|743082
dtype: object

In [104]:
segment3_workflow3=pd.read_csv('segment3_workflowS3.csv')
segment3_workflow3.CASE_ID.mode()

0    1102908|470757|722758
dtype: object

In [283]:
segment3_workflow4=pd.read_csv('segment3_workflowS4.csv')
segment3_workflow4.CASE_ID.mode()

0    1052275|402830|625246
dtype: object

In [4]:
segment3_workflow1.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Submit,Submit,26/11/2018 17:58:06,GUEST,Student,15,N,1. App to Assessment,Part 1 - Application ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,AR - Satisfied - previous or current study,AR - Satisfied - Academic results for prior te...,7/12/2018 10:51:18,ATSCHORN,Academic Services,14,N,1. App to Assessment,Part 6 - Requirement Verify,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,AR - Not Satisfied - ELR,AR - Not Satisfied - English language proficiency,7/12/2018 10:51:38,ATSCHORN,Academic Services,13,N,1. App to Assessment,Part 6 - Requirement Verify,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,1067449|424909|657898,OC - Created - Other,OC - Created - English language requirements,7/12/2018 10:51:46,ATSCHORN,Academic Services,12,N,1. App to Assessment,Part 5 - Requirement Create,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
4,1067449|424909|657898,AR - Created - Other,AR - Created - Assessment information (Coursew...,7/12/2018 11:00:43,ATSCHORN,Academic Services,11,N,1. App to Assessment,Part 5 - Requirement Create,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y


In [5]:
segment3_workflow2.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Send for Assessment,Send for Assessment,7/12/2018 11:01:50,1067449,NaN,9,N,1. App to Assessment,Part 1 - Application ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Decision - Qualified,14/12/2018 15:55:39,RGORDON1,Academic,8,N,2. Perform Assessment,Part 4 - Assessment Decision,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,Assessment Outcome - Successful,Assessment Outcome - Successful,18/12/2018 11:34:06,KRIT,Faculty,7,N,2. Perform Assessment,Part 3 - Assessment Outcome,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,913640|554685|848410,Send for Assessment,Send for Assessment,3/12/2019 11:06:46,913640,NaN,1,Y,1. App to Assessment,Part 1 - Application ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,NaN,NaN,NaN
4,1021374|367597|573577,Send for Assessment,Send for Assessment,26/6/2018 11:26:41,1021374,NaN,17,N,1. App to Assessment,Part 1 - Application ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N


In [105]:
segment3_workflow3.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,REFERRAL_ID,APPLICATION_ID,APPLICATION_LINE_ID,STU_ID,STU_CITIZEN_CD,STU_CITIZEN_NM,STU_BIRTH_DT,STU_ABOR_TSI_CD,STU_ABOR_TSI_NM,STU_BIRTH_CNTRY_CD,STU_BIRTH_CNTRY_NM,STU_CTZN_CNTRY_CD,STU_CTZN_CNTRY_NM,SUBMISSION_METHOD,ENTRY_TYPE,ASSESSMENT_DECISION_SCORE,ASSESSMENT_DECISION_NM,ASSESSMENT_OUTCOME_NM,LAST_COUNTRY_EDUCATION,PREV_INST_NM_MAIN,PREV_INST_NM_ALT1,PREV_INST_NM_ALT2,PREFERENCE_NO,SPK_CD,SPK_FULL_TITLE,ORG_UNIT_CD,AVAIL_ORG_UNIT_NM,SUPPORTING_ORG,STUDY_MODE_CD,RETURNING_DEFERRAL,CURR_AVAIL_YR,CURR_STUDY_PERIOD,CURR_SPRD_CD,CURR_SPRD_NM,ORIG_AVAIL_YR,ORIG_STUDY_PERIOD,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Assessment Outcome - Successful,Assessment Outcome - Successful,18/12/2018 11:34:06,KRIT,Faculty,7,N,2. Perform Assessment,Part 3 - Assessment Outcome,NaN,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,Create Offer,Create Offer,18/12/2018 11:34:51,KRIT,Faculty,6,N,3b. Offer stage,Part 2 - Application Line ID Workflow,NaN,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,Release Offer,Release Offer,18/12/2018 11:36:07,KRIT,Faculty,5,N,3b. Offer stage,Part 2 - Application Line ID Workflow,NaN,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,1021374|367597|573577,Assessment Outcome - Successful,Assessment Outcome - Successful,12/7/2018 10:07:14,ATKINSONM,Faculty,12,N,2. Perform Assessment,Part 3 - Assessment Outcome,NaN,367597,573577,1021374,2,New Zealander,17/9/1995 0:00:00,N,No,1201.0,New Zealand,1201.0,New Zealand,Online Application,Bachelor (equiv Aust 3yr deg),71.3,Qualified,Successful,New Zealand,"Auckland, University of",NaN,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N
4,1021374|367597|573577,Create Offer,Create Offer,12/7/2018 10:07:30,ATKINSONM,Faculty,11,N,3b. Offer stage,Part 2 - Application Line ID Workflow,NaN,367597,573577,1021374,2,New Zealander,17/9/1995 0:00:00,N,No,1201.0,New Zealand,1201.0,New Zealand,Online Application,Bachelor (equiv Aust 3yr deg),71.3,Qualified,Successful,New Zealand,"Auckland, University of",NaN,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N


In [284]:
segment3_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,REFERRAL_ID,APPLICATION_ID,APPLICATION_LINE_ID,STU_ID,STU_CITIZEN_CD,STU_CITIZEN_NM,STU_BIRTH_DT,STU_ABOR_TSI_CD,STU_ABOR_TSI_NM,STU_BIRTH_CNTRY_CD,STU_BIRTH_CNTRY_NM,STU_CTZN_CNTRY_CD,STU_CTZN_CNTRY_NM,SUBMISSION_METHOD,ENTRY_TYPE,ASSESSMENT_DECISION_SCORE,ASSESSMENT_DECISION_NM,ASSESSMENT_OUTCOME_NM,LAST_COUNTRY_EDUCATION,PREV_INST_NM_MAIN,PREV_INST_NM_ALT1,PREV_INST_NM_ALT2,PREFERENCE_NO,SPK_CD,SPK_FULL_TITLE,ORG_UNIT_CD,AVAIL_ORG_UNIT_NM,SUPPORTING_ORG,STUDY_MODE_CD,RETURNING_DEFERRAL,CURR_AVAIL_YR,CURR_STUDY_PERIOD,CURR_SPRD_CD,CURR_SPRD_NM,ORIG_AVAIL_YR,ORIG_STUDY_PERIOD,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Release Offer,Release Offer,18/12/2018 11:36:07,KRIT,Faculty,5,N,3b. Offer stage,Part 2 - Application Line ID Workflow,NaN,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,OC - Satisfied - Other,OC - Satisfied - English language requirements,25/1/2019 15:18:53,KNAVIN,Academic Services,4,N,1. App to Assessment,Part 6 - Requirement Verify,Other,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,Accept,Accept,6/2/2019 9:12:01,KRIT,Faculty,3,N,4. Post offer,Part 2 - Application Line ID Workflow,NaN,424909,657898,1067449,1,Australian,22/7/1968 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,Masters CW,0.0,Qualified,Successful,NaN,NaN,NaN,NaN,1,MC-ENGYSYS,Master of Energy Systems,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,1021374|367597|573577,Release Offer,Release Offer,12/7/2018 10:10:52,ATKINSONM,Faculty,10,N,3b. Offer stage,Part 2 - Application Line ID Workflow,NaN,367597,573577,1021374,2,New Zealander,17/9/1995 0:00:00,N,No,1201.0,New Zealand,1201.0,New Zealand,Online Application,Bachelor (equiv Aust 3yr deg),71.3,Qualified,Successful,New Zealand,"Auckland, University of",NaN,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N
4,1021374|367597|573577,Accept,Accept,8/8/2018 12:14:54,ATKINSONM,Faculty,9,N,4. Post offer,Part 2 - Application Line ID Workflow,NaN,367597,573577,1021374,2,New Zealander,17/9/1995 0:00:00,N,No,1201.0,New Zealand,1201.0,New Zealand,Online Application,Bachelor (equiv Aust 3yr deg),71.3,Qualified,Successful,New Zealand,"Auckland, University of",NaN,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,N,2019,Start Year Intake,STYI,Start Year Intake,2019,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N


In [8]:
segment3_workflow1['WRKFLW_TSK_MAP_DETAILED'].nunique()

112

In [6]:
segment3_workflow1=segment3_workflow1.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow1.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Submit
1,1067449|424909|657898,AR - Satisfied - Academic results for prior te...
2,1067449|424909|657898,AR - Not Satisfied - English language proficiency
3,1067449|424909|657898,OC - Created - English language requirements
4,1067449|424909|657898,AR - Created - Assessment information (Coursew...


In [7]:
segment3_workflow2=segment3_workflow2.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow2.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Send for Assessment
1,1067449|424909|657898,Acacdemic - Assessment Decision - Qualified
2,1067449|424909|657898,Assessment Outcome - Successful
3,913640|554685|848410,Send for Assessment
4,1021374|367597|573577,Send for Assessment


In [106]:
segment3_workflow3=segment3_workflow3.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow3.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Assessment Outcome - Successful
1,1067449|424909|657898,Create Offer
2,1067449|424909|657898,Release Offer
3,1021374|367597|573577,Assessment Outcome - Successful
4,1021374|367597|573577,Create Offer


In [285]:
segment3_workflow4=segment3_workflow4.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Release Offer
1,1067449|424909|657898,OC - Satisfied - English language requirements
2,1067449|424909|657898,Accept
3,1021374|367597|573577,Release Offer
4,1021374|367597|573577,Accept


In [286]:
numOfWorkflowProcesses=segment3_workflow1.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses.head()

CASE_ID
1000727|431447|668147     3
1001829|336678|526826     3
1001829|336678|526827     3
1001877|336757|526956     4
1002271|337186|527523    10
Name: CASE_ID, dtype: int64

In [287]:
numOfWorkflowProcesses2=segment3_workflow2.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses2.head()

CASE_ID
1000727|431447|668147    11
1001829|336678|526826     3
1001829|336678|526827     3
1001877|336757|526956    11
1002271|337186|527523     8
Name: CASE_ID, dtype: int64

In [288]:
numOfWorkflowProcesses3=segment3_workflow3.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses3.head()

CASE_ID
1000727|431447|668147    4
1001829|336678|526826    3
1001877|336757|526956    3
1002271|337186|527523    3
1007394|346044|540306    3
Name: CASE_ID, dtype: int64

In [289]:
numOfWorkflowProcesses4=segment3_workflow4.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses4.head()

CASE_ID
1000727|431447|668147    2
1001829|336678|526826    2
1001877|336757|526956    1
1002271|337186|527523    2
1007394|346044|540306    2
Name: CASE_ID, dtype: int64

In [290]:
counts = segment3_workflow1['CASE_ID'].value_counts()

In [291]:
counts2 = segment3_workflow2['CASE_ID'].value_counts()

In [292]:
counts3 = segment3_workflow3['CASE_ID'].value_counts()

In [293]:
counts4 = segment3_workflow4['CASE_ID'].value_counts()

In [294]:
counts.head()

1175521|568653|866936    19
1075305|437160|675799    18
1125321|497557|764055    17
1086364|448364|689547    17
1045292|395605|615665    17
Name: CASE_ID, dtype: int64

In [295]:
counts2.head()

1114684|484215|743082    22
1028582|370276|577743    16
1076583|437721|676610    15
1019820|360156|561476    15
1025815|366554|572153    15
Name: CASE_ID, dtype: int64

In [296]:
counts3.head()

1102908|470757|722758    12
1147799|528824|809849    11
1025815|366554|572153    10
1140264|517887|794176     9
1120436|490900|754192     9
Name: CASE_ID, dtype: int64

In [297]:
counts4.head()

1052275|402830|625246    7
1025305|366531|571922    6
954468|360490|561961     6
1152919|538301|823869    6
354059|390663|608273     5
Name: CASE_ID, dtype: int64

In [442]:
filtered = counts[counts > 12] 
seg3_count1=segment3_workflow1[segment3_workflow1['CASE_ID'].isin(filtered.index)]

In [443]:
filtered2 = counts2[counts2 > 12]
seg3_count2=segment3_workflow2[segment3_workflow2['CASE_ID'].isin(filtered.index)]

In [444]:
filtered3 = counts3[counts3 > 12]
seg3_count3=segment3_workflow3[segment3_workflow3['CASE_ID'].isin(filtered.index)]

In [445]:
filtered4 = counts4[counts4 > 12]
seg3_count4=segment3_workflow4[segment3_workflow4['CASE_ID'].isin(filtered.index)]

In [446]:
seg3_count1['CASE_ID'].nunique()

37

In [447]:
seg3_count1.head(14)

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
20,1128107|501518|769785,Submit
21,1128107|501518|769785,AR - Satisfied - Official syllabus or subject ...
22,1128107|501518|769785,AR - Satisfied - Advanced Standing
23,1128107|501518|769785,AR - Satisfied - Specialisation (Master of Inf...
24,1128107|501518|769785,AR - Satisfied - Graduate Access Melbourne
25,1128107|501518|769785,AR - Satisfied - Current enrolment at the Univ...
26,1128107|501518|769785,AR - Satisfied - Entry points (Master of Infor...
27,1128107|501518|769785,AR - Satisfied - Academic results for prior te...
28,1128107|501518|769785,AR - Satisfied - Transcript û My eQuals
29,1128107|501518|769785,AR - Created - Further information (1)


In [448]:
seg3_count2['CASE_ID'].nunique()

30

In [449]:
seg3_count2.head(17)

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
10,1128107|501518|769785,Send for Assessment
11,1128107|501518|769785,Assessment Outcome - Not Successful
59,1086364|448364|689547,Send for Assessment
60,1086364|448364|689547,Assessment Decision - No Decision
61,1086364|448364|689547,Acacdemic - Assessment Decision - Not Qualified
62,1086364|448364|689547,Assessment Outcome - Successful
146,1101054|468718|719852,Send for Assessment
147,1101054|468718|719852,Acacdemic - Assessment Decision - Qualified
148,1101054|468718|719852,End My Assessment
149,1101054|468718|719852,Assessment Outcome - Not Successful


In [450]:
seg3_count3['CASE_ID'].nunique()

27

In [451]:
seg3_count3.head(17)

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
6,1128107|501518|769785,Assessment Outcome - Not Successful
24,1086364|448364|689547,Assessment Outcome - Successful
25,1086364|448364|689547,Create Offer
26,1086364|448364|689547,Release Offer
50,1072839|432324|669486,Assessment Outcome - Successful
51,1072839|432324|669486,Create Offer
52,1072839|432324|669486,Release Offer
70,1101054|468718|719852,Assessment Outcome - Not Successful
71,1101054|468718|719852,Assessment Decision - No Decision
97,84841|418611|648829,Assessment Outcome - Successful


In [452]:
seg3_count4['CASE_ID'].nunique()

20

In [453]:
seg3_count4.head(17)

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
12,1086364|448364|689547,Release Offer
13,1086364|448364|689547,AR - Satisfied - Official syllabus or subject ...
26,1072839|432324|669486,Release Offer
27,1072839|432324|669486,Release Triage Hold
28,1072839|432324|669486,Send for Assessment
29,1072839|432324|669486,Accept
50,84841|418611|648829,Release Offer
51,84841|418611|648829,Accept
52,1101040|468693|719818,Release Offer
53,1101040|468693|719818,Assessment Decision - Qualified


In [454]:
frames = [seg3_count1, seg3_count2, seg3_count3, seg3_count4]
count_12=pd.concat(frames)

In [455]:
y = pd.get_dummies(seg3_count1.WRKFLW_TSK_MAP_DETAILED)
y=y[:0]

In [456]:
y

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1),AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Assessment Outcome - Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Offer,Release Triage Hold,Send for Assessment,Submit,WITHDRAW & LOCK


In [457]:
y2 = pd.get_dummies(seg3_count2.WRKFLW_TSK_MAP_DETAILED)
y2=y2[:0]

In [458]:
y2

,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - English language proficiency,AR - Satisfied - Further information (1),Acacdemic - Assessment Decision - Not Qualified,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Assessment Outcome - Not Successful,Assessment Outcome - Successful,Back to Triage,Create Offer,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment


In [459]:
y3=pd.merge(y, y2, on='Send for Assessment')
y3

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies_x,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency_x,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1)_x,AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified_x,Assessment Decision - No Decision_x,Assessment Decision - Not Qualified_x,Assessment Decision - Qualified_x,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful_x,Assessment Outcome - Successful_x,Create Offer_x,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Offer,Release Triage Hold,Submit,WITHDRAW & LOCK,AR - Satisfied - Academic results for prior tertiary studies_y,AR - Satisfied - English language proficiency_y,AR - Satisfied - Further information (1)_y,Acacdemic - Assessment Decision - Not Qualified_y,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Decision - No Decision_y,Assessment Decision - Not Qualified_y,Assessment Decision - Qualified_y,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Assessment Outcome - Not Successful_y,Assessment Outcome - Successful_y,Back to Triage,Create Offer_y,End 

In [460]:
y3

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies_x,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency_x,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1)_x,AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified_x,Assessment Decision - No Decision_x,Assessment Decision - Not Qualified_x,Assessment Decision - Qualified_x,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful_x,Assessment Outcome - Successful_x,Create Offer_x,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Offer,Release Triage Hold,Submit,WITHDRAW & LOCK,AR - Satisfied - Academic results for prior tertiary studies_y,AR - Satisfied - English language proficiency_y,AR - Satisfied - Further information (1)_y,Acacdemic - Assessment Decision - Not Qualified_y,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Decision - No Decision_y,Assessment Decision - Not Qualified_y,Assessment Decision - Qualified_y,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Assessment Outcome - Not Successful_y,Assessment Outcome - Successful_y,Back to Triage,Create Offer_y,End 

In [461]:
y3.columns = y3.columns.str.replace("_x", "")

In [462]:
y3.columns = y3.columns.str.replace("_y", "")

In [463]:
y3 = y3.loc[:,~y3.columns.duplicated()]

In [464]:
y3.shape

(0, 78)

In [465]:
y4 = pd.get_dummies(seg3_count3.WRKFLW_TSK_MAP_DETAILED)
y4=y4[:0]

In [466]:
y4

,AR - Satisfied - English language proficiency,AR - Satisfied - English language requirements,AR - Satisfied - Further information (1),Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - Not Successful,Assessment Outcome - Successful,Create Offer,End My Assessment,OC - Created - Evidence of course completion,PREPARE TO LOCK,Release Offer,Release Triage Hold,Send for Assessment


In [467]:
y5=pd.merge(y3, y4, on='Assessment Outcome - Successful')
y5

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency_x,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1)_x,AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision_x,Assessment Decision - Not Qualified_x,Assessment Decision - Qualified_x,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful_x,Create Offer_x,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Offer_x,Release Triage Hold_x,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment_x,OC - Created - Evidence of course completion_x,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment_x,AR - Satisfied - English language proficiency_y,AR - Satisfied - English language requirements,AR - Satisfied - Further information (1)_y,Assessment Decision - No Decision_y,Assessment Decision - Not Qualified_y,Assessment Decision - Qualified_y,Assessme

In [468]:
y5.columns = y5.columns.str.replace("_x", "")

In [469]:
y5.columns = y5.columns.str.replace("_y", "")

In [470]:
y5 = y5.loc[:,~y5.columns.duplicated()]

In [471]:
y5.shape

(0, 80)

In [472]:
y6 = pd.get_dummies(seg3_count4.WRKFLW_TSK_MAP_DETAILED)
y6=y6[:0]

In [473]:
y7=pd.merge(y5, y6, on='Release Offer')
y7

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1)_x,AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English)_x,AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified_x,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements_x,Put Triage On-Hold,Release Triage Hold_x,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion_x,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment_x,AR - Satisfied - English language requirements,Assessment Outcome - Successful,PREPARE TO LOCK_x,AR - Satisfied - Further information (1)_y,AR - Satisfied - Official syllabus or subject descriptions (in English)_y,Accept,Assessment Decision - Qualified_y,Decline,Manage Acceptan

In [474]:
y7.columns = y7.columns.str.replace("_x", "")

In [475]:
y7.columns = y7.columns.str.replace("_y", "")

In [476]:
y7 = y7.loc[:,~y7.columns.duplicated()]

In [477]:
y7.shape

(0, 84)

In [478]:
y7["CASE_ID"]=""

In [479]:
col_name="CASE_ID"
first_col = y7.pop(col_name)

In [480]:
y7.insert(0, col_name, first_col)

In [481]:
y7['CASE_ID']=count_12['CASE_ID'].unique()

<ipython-input-481-15a10ebd34fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y7['CASE_ID']=count_12['CASE_ID'].unique()


In [482]:
import numpy as np
y7 = y7.replace(np.nan,0, regex=True)
y7.head()

,CASE_ID,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1),AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Triage Hold,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment,AR - Satisfied - English language requirements,Assessment Outcome - Successful,PREPARE TO LOCK,Accept,Decline,Manage Acceptance,OC - Not Satisfied - Evidence of course completion,Release Offer
0,1128107|501518|769785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [483]:
count_12.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
20,1128107|501518|769785,Submit
21,1128107|501518|769785,AR - Satisfied - Official syllabus or subject ...
22,1128107|501518|769785,AR - Satisfied - Advanced Standing
23,1128107|501518|769785,AR - Satisfied - Specialisation (Master of Inf...
24,1128107|501518|769785,AR - Satisfied - Graduate Access Melbourne


In [484]:
dict_1=count_12.to_dict(orient='list')

In [485]:
case_id = dict_1['CASE_ID']

In [486]:
workflow = dict_1['WRKFLW_TSK_MAP_DETAILED']

In [487]:
print(type(workflow))
print(len(workflow))

<class 'list'>
767


In [488]:
from collections import defaultdict

case_wf_dict = defaultdict(list)

for i in range(len(case_id)):
    item = case_id[i]
    case_wf_dict[item].append(workflow[i])
print(len(case_wf_dict))

37


In [489]:
y7 = y7.set_index('CASE_ID')

In [490]:
y7.head()

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1),AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Triage Hold,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment,AR - Satisfied - English language requirements,Assessment Outcome - Successful,PREPARE TO LOCK,Accept,Decline,Manage Acceptance,OC - Not Satisfied - Evidence of course completion,Release Offer
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1128107

In [491]:
y_col_name = y7.columns.values.tolist()
print(y_col_name)

['AR - Created - Academic results for prior tertiary studies', 'AR - Created - Academic transcript Grading Scheme', 'AR - Created - Academic transcripts for prior tertiary studies', 'AR - Created - Academic transcripts statement of completion', 'AR - Created - Advanced standing application', 'AR - Created - Assessment information (Coursework) (1)', 'AR - Created - Curriculum Vitae', 'AR - Created - English language requirements', 'AR - Created - Further information (1)', 'AR - Created - Further information (2)', 'AR - Created - Further information - Transcript translation and original language', 'AR - Created - Further information - Transcripts for subject credited from previous study', 'AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications', 'AR - Created - Grading scale', 'AR - Created - Graduate Access Melbourne', 'AR - Created - Syllabus (Melbourne School of Engineering)', 'AR - Created - University of Melbourne Statement of Re

In [492]:
for key in case_wf_dict.keys():
    for process in case_wf_dict[key]:
        if process in y_col_name:
            y7.loc[key, process] = 1
        else:
            continue

In [493]:
pd.set_option('display.max_columns', None)

In [494]:
y7['Submit'].head()

CASE_ID
1128107|501518|769785    1.0
1086364|448364|689547    1.0
1072839|432324|669486    1.0
1101054|468718|719852    1.0
84841|418611|648829      1.0
Name: Submit, dtype: float64

In [495]:
y7['Send for Assessment'].head()

CASE_ID
1128107|501518|769785    1.0
1086364|448364|689547    1.0
1072839|432324|669486    1.0
1101054|468718|719852    1.0
84841|418611|648829      1.0
Name: Send for Assessment, dtype: float64

In [496]:
y7.head()

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1),AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Triage Hold,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment,AR - Satisfied - English language requirements,Assessment Outcome - Successful,PREPARE TO LOCK,Accept,Decline,Manage Acceptance,OC - Not Satisfied - Evidence of course completion,Release Offer
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1128107

In [497]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [498]:
y7.head()

,AR - Created - Academic results for prior tertiary studies,AR - Created - Academic transcript Grading Scheme,AR - Created - Academic transcripts for prior tertiary studies,AR - Created - Academic transcripts statement of completion,AR - Created - Advanced standing application,AR - Created - Assessment information (Coursework) (1),AR - Created - Curriculum Vitae,AR - Created - English language requirements,AR - Created - Further information (1),AR - Created - Further information (2),AR - Created - Further information - Transcript translation and original language,AR - Created - Further information - Transcripts for subject credited from previous study,AR - Created - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Created - Grading scale,AR - Created - Graduate Access Melbourne,AR - Created - Syllabus (Melbourne School of Engineering),AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Employer reference letter,AR - Requested - Further information (1),AR - Reviewed - Graduate Access Melbourne,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Appointment of authorised representative,AR - Satisfied - Assessment information (Coursework) (1),AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - Employer reference letter,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information (1),AR - Satisfied - Further information (2),AR - Satisfied - Further information - Transcript translation and original language,AR - Satisfied - Further information - Transcripts for subject credited from previous study,AR - Satisfied - Further information û Preference order required as submitted multiple 1st prefer applications,AR - Satisfied - Grading scale,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),AR - Satisfied - Personal statement (500 words),AR - Satisfied - Specialisation (Master of Information Systems),AR - Satisfied - Specialisation (Master of Information Technology),AR - Satisfied - Syllabus (Melbourne School of Engineering),AR - Satisfied - Transcript û CHESICC,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Acacdemic - Assessment Decision - Not Qualified,Assessment Decision - No Decision,Assessment Decision - Not Qualified,Assessment Decision - Qualified,Assessment Outcome - No Outcome Set,Assessment Outcome - Not Successful,Create Offer,OC - Created - CHESICC transcript authentication for international students,OC - Created - English language requirements,OC - Reviewed - CHESICC transcript authentication for international students,OC - Reviewed - English language requirements,OC - Satisfied - English language requirements,Put Triage On-Hold,Release Triage Hold,Submit,WITHDRAW & LOCK,Acacdemic - Assessment Decision - Qualified,Acacdemic - Assessment Outcome - Successful,Assessment Outcome - Higher Preference Successful,Assessment Outcome - Ineligible / Withdrawn,Back to Triage,End My Assessment,OC - Created - Evidence of course completion,OC - Satisfied - Evidence of course completion,Put Assessment On-Hold,Release Assessment Hold,Send for Assessment,AR - Satisfied - English language requirements,Assessment Outcome - Successful,PREPARE TO LOCK,Accept,Decline,Manage Acceptance,OC - Not Satisfied - Evidence of course completion,Release Offer
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1128107

In [499]:
y7 = movecol(y7, 
             cols_to_move=['Accept'], 
             ref_col='Release Offer',
             place='After')

In [501]:
y7.shape

(37, 84)

In [502]:
y7=y7.fillna(0.0)

In [504]:
X=y7.iloc[:,0:83].values
Y=y7.iloc[:,83].values 

In [505]:
#spliting dataset
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size = 0.3,random_state=0)

In [506]:
# K Nearest Neighbors
Knn = KNeighborsClassifier(n_neighbors=5)
Knn.fit(x_train,y_train)

KNeighborsClassifier()

In [507]:
knn_predict=Knn.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, knn_predict)*100,2)))

accuracy: 75.0%
